In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
print 'march_import'

In [ ]:
# utils
## chinese decode 
def cn(s):
#     return s.decode('utf8')
    return s
## 字符串类型转换为类别的数字编码
dict_short_url  = {'init':0} #和短地址一样，统一记录使用过的int
dict_short_url_index = {0: 'init'}
def getShortUrl(s):
    # 未记录，则添加记录
    if dict_short_url.has_key(s) == False:
        dict_short_url[s] = max(dict_short_url.values())+1
        dict_short_url_index[dict_short_url[s]] = s
    return dict_short_url[s]

In [ ]:
# read data
# trainData means rawdata, X_train means cross_validation data
trainData = pd.merge(pd.read_csv('/data/topic1/ensemble_train_x.csv'), pd.read_csv('/data/topic1/ensemble_train_y.csv'), on='用户标识')
testData = pd.read_csv('/data/topic1/ensemble_test_x.csv')
print 'march_load_data'

In [ ]:
# 读入外部特征
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countapppv.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countapp.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countoutprovince.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countwebpv.csv'), on='用户标识')
trainData = pd.merge(trainData, pd.read_csv('/data/topic1/feature_countweb.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_countapppv.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_countapp.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_countoutprovince.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_countwebpv.csv'), on='用户标识')
testData = pd.merge(testData, pd.read_csv('/data/topic1/feature_countweb.csv'), on='用户标识')
print 'march_external_feature'

In [ ]:
# 内部特征
trainData['phone_brand'] = trainData[cn('手机品牌')].apply(getShortUrl)
trainData['phone_model_number'] = trainData[cn('手机终端型号')].apply(getShortUrl)
trainData['if_cross_province'] = trainData[cn('是否有跨省行为')].apply(getShortUrl)
trainData['if_cross_conuntry'] = trainData[cn('是否有出境行为')].apply(getShortUrl)
testData['phone_brand'] = testData[cn('手机品牌')].apply(getShortUrl)
testData['phone_model_number'] = testData[cn('手机终端型号')].apply(getShortUrl)
testData['if_cross_province'] = testData[cn('是否有跨省行为')].apply(getShortUrl)
testData['if_cross_conuntry'] = testData[cn('是否有出境行为')].apply(getShortUrl)
trainData.drop([cn('手机品牌'), cn('是否有跨省行为'), cn('是否有出境行为'), cn('漫入省份'), cn('漫出省份'), cn('手机终端型号')], axis=1, inplace=True)
testData.drop([cn('手机品牌'), cn('是否有跨省行为'), cn('是否有出境行为'), cn('漫入省份'), cn('漫出省份'), cn('手机终端型号')], axis=1, inplace=True)
print 'march_inner_feature'

## 数据预处理 

In [ ]:
# 缺失值处理
trainData.fillna(0, inplace=True)
testData.fillna(0, inplace=True)
print 'march_data_done'

## 模型训练

In [ ]:
# 修改列名， lgb不支持中文列名
dict_new_columns = {}
for i in trainData.drop([cn('用户标识'),cn('是否去过迪士尼')],axis=1).columns:
    dict_new_columns[i] = str(getShortUrl(i))
trainData.rename(columns=dict_new_columns, inplace = True)
testData.rename(columns=dict_new_columns, inplace = True)

In [ ]:
# 声明类别变量
temp_categorical_feature = [cn('性别'),cn('大致消费水平'),'phone_brand','phone_model_number','if_cross_province','if_cross_conuntry']
list_categorical_feature = [str(dict_short_url[item]) for item in temp_categorical_feature]

In [ ]:
print 'march_init_model'
train_data = lgb.Dataset(trainData.drop([cn('用户标识'),cn('是否去过迪士尼')],axis=1).values, label=trainData[cn('是否去过迪士尼')], feature_name=list(trainData.drop([cn('用户标识'),cn('是否去过迪士尼')],axis=1).columns), categorical_feature=list_categorical_feature)
param = {'num_leaves':63, 'num_trees':1000, 'objective':'binary', 'metric':'auc'}
# bst=lgb.cv(param, train_data, num_boost_round=100, nfold=3, early_stopping_rounds=30)
# bst = lgb.train(param, train_data, num_boost_round=100)
bst = lgb.train(param, train_data, num_boost_round=69)
ypred = bst.predict(testData.drop([cn('用户标识')],axis=1).values)
print 'march_model_done'

In [ ]:
# 输出csv结果
output = pd.DataFrame({'IMEI': testData[cn('用户标识')].values,'SCORE':ypred})
# output.to_csv('E:\\work\\wo+\\data\\1\\result.csv', index=False, float_format='%.5f')
output.to_csv('/data/topic1/ensemble_result_lgb.csv', index=False, float_format='%.5f')